## Read files

In [1]:
def read_file(file):
    with open(file) as f:
        text = f.read()
    return text    

In [2]:
doc = read_file('moby_dick_four_chapters.txt')
len(doc)

61614

## Tokenize, clean text

In [3]:
import spacy 

nlp = spacy.load('en_core_web_md', disable=['parser', 'tagger', 'ner'])
nlp.max_length = 1198623

In [4]:
def tokenize_text(doc):
    return [token.text.lower() for token in nlp(doc) if token.text not in '\n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n ']

text = tokenize_text(doc)
len(text)

C:\Users\Atishay Bajpai\Anaconda3\lib\site-packages\spacy\pipeline\lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


11338

In [227]:
#25 words to generate -> 26th word using NN

In [6]:
text_seq = []
train_len = 25+1

for i in range(train_len, len(text)):
    seq = text[i-train_len:i]
    text_seq.append(seq)

text_seq[0]

['call',
 'me',
 'ishmael',
 'some',
 'years',
 'ago',
 'never',
 'mind',
 'how',
 'long',
 'precisely',
 'having',
 'little',
 'or',
 'no',
 'money',
 'in',
 'my',
 'purse',
 'and',
 'nothing',
 'particular',
 'to',
 'interest',
 'me',
 'on']

In [7]:
len(text_seq)

11312

In [8]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_seq)

sequences = tokenizer.texts_to_sequences(text_seq)

sequences[0]

[956,
 14,
 263,
 51,
 261,
 408,
 87,
 219,
 129,
 111,
 954,
 260,
 50,
 43,
 38,
 314,
 7,
 23,
 546,
 3,
 150,
 259,
 6,
 2713,
 14,
 24]

In [232]:
tokenizer.index_word[956]


'call'

In [10]:
vocabulary_size = len(tokenizer.word_counts) # or len(tokenizer.index_word). Anything
vocabulary_size

2718

In [11]:
import numpy as np

sequences = np.array(sequences)
sequences

array([[ 956,   14,  263, ..., 2713,   14,   24],
       [  14,  263,   51, ...,   14,   24,  957],
       [ 263,   51,  261, ...,   24,  957,    5],
       ...,
       [ 952,   12,  166, ...,  262,   53,    2],
       [  12,  166, 2712, ...,   53,    2, 2718],
       [ 166, 2712,    3, ...,    2, 2718,   26]])

In [12]:
# from keras.utils import to_categorical 
from keras.utils import np_utils
# .to_categorical(y_train, num_classes)
# sequences[:,:-1]

In [13]:
X = sequences[:,:-1]
y = sequences[:,-1]

y = np_utils.to_categorical(y, num_classes=vocabulary_size+1)

In [14]:
seq_len = X.shape[1]
seq_len

25

In [15]:
import numpy as np

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
# import keras
# from keras.models import Sequential
# from keras.layers import Dense,LSTM,Embedding

In [25]:
def create_model(vocabulary_size, seq_len):
    model = Sequential()
    model.add(Embedding(vocabulary_size, 25, input_length=seq_len))
    model.add(LSTM(100, return_sequences=True))
    model.add(LSTM(100))
    model.add(Dense(100, activation='relu'))

    model.add(Dense(vocabulary_size, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
   
    model.summary()
    
    return model

In [26]:
model = create_model(vocabulary_size+1, seq_len)
model

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 25, 25)            67975     
                                                                 
 lstm_4 (LSTM)               (None, 25, 100)           50400     
                                                                 
 lstm_5 (LSTM)               (None, 100)               80400     
                                                                 
 dense_4 (Dense)             (None, 100)               10100     
                                                                 
 dense_5 (Dense)             (None, 2719)              274619    
                                                                 
Total params: 483,494
Trainable params: 483,494
Non-trainable params: 0
_________________________________________________________________


In [27]:
from pickle import dump, load

In [28]:
model.fit(X,y,batch_size=256,epochs=300, verbose=1)

Epoch 1/300
45/45 [==============================] - 16s 206ms/step - loss: 7.1035 - accuracy: 0.0433
Epoch 2/300
45/45 [==============================] - 10s 217ms/step - loss: 6.4030 - accuracy: 0.0529
Epoch 3/300
45/45 [==============================] - 10s 219ms/step - loss: 6.3530 - accuracy: 0.0529
Epoch 4/300
45/45 [==============================] - 9s 189ms/step - loss: 6.3395 - accuracy: 0.0529
Epoch 5/300
45/45 [==============================] - 6s 125ms/step - loss: 6.3345 - accuracy: 0.0529
Epoch 6/300
45/45 [==============================] - 6s 132ms/step - loss: 6.3236 - accuracy: 0.0529
Epoch 7/300
45/45 [==============================] - 7s 163ms/step - loss: 6.2511 - accuracy: 0.0529
Epoch 8/300
45/45 [==============================] - 7s 149ms/step - loss: 6.1441 - accuracy: 0.0529
Epoch 9/300
45/45 [==============================] - 6s 129ms/step - loss: 6.0929 - accuracy: 0.0529
Epoch 10/300
45/45 [==============================] - 6s 124ms/step - loss: 6.0394 - acc

45/45 [==============================] - 7s 147ms/step - loss: 2.6668 - accuracy: 0.3727
Epoch 161/300
45/45 [==============================] - 7s 145ms/step - loss: 2.6465 - accuracy: 0.3765
Epoch 162/300
45/45 [==============================] - 6s 144ms/step - loss: 2.6332 - accuracy: 0.3798
Epoch 163/300
45/45 [==============================] - 7s 146ms/step - loss: 2.6261 - accuracy: 0.3834
Epoch 164/300
45/45 [==============================] - 6s 144ms/step - loss: 2.6121 - accuracy: 0.3790
Epoch 165/300
45/45 [==============================] - 6s 143ms/step - loss: 2.6047 - accuracy: 0.3879
Epoch 166/300
45/45 [==============================] - 7s 151ms/step - loss: 2.5890 - accuracy: 0.3874
Epoch 167/300
45/45 [==============================] - 7s 146ms/step - loss: 2.5885 - accuracy: 0.3853
Epoch 168/300
45/45 [==============================] - 6s 145ms/step - loss: 2.5768 - accuracy: 0.3874
Epoch 169/300
45/45 [==============================] - 6s 144ms/step - loss: 2.5580 - a

In [29]:
model.save('text_gen_model.h5')

In [30]:
dump(tokenizer, open('simple_tokenizer', 'wb'))

In [83]:
from keras.preprocessing.sequence import pad_sequences

def generate_text(model, tokenizer, seq_len, seed_text, num_gen_words):
    
    output_text = [] 
    input_text = seed_text
    
    for i in range(num_gen_words):
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]
        pad_encoding = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
        pred_word_ind = np.argmax(model.predict(pad_encoding), axis=-1)[0] #model.predict_classes(pad_encoding, verbose=0)[0]
        pred_word = tokenizer.index_word[pred_word_ind]
        input_text = input_text+ ' ' + pred_word
        output_text.append(pred_word)
        final_text = input_text
    return ' '.join(output_text)

In [162]:
import random
random.seed()
random_pick = random.randint(0,len(text_seq))
random_pick
random_seed_text = text_seq[random_pick]
seed_text = ' '.join(random_seed_text)
seed_text
# seed_text = 'mean to have it inferred that i ever go to sea as a passenger for to go as a passenger you must needs have a'

'hugged me tightly as though naught but death should part us twain i now strove to rouse him--"queequeg!"--but his only answer was a snore i then'

In [163]:
generate_text(model, tokenizer, seq_len, seed_text=seed_text, num_gen_words=10)

"did n't stop said a thirty years ' war and"